<a href="https://colab.research.google.com/github/raulvalerio/Titanic-with-Python/blob/master/WalmartUnitSale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT AND READ DATASETS

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
#train = pd.read_csv('C:/Users/isach/Documents/Kaggle Competitions/Walmart Unit Sales/sales_train_validation.csv')
calendar = pd.read_csv('C:/Users/isach/Documents/Kaggle Competitions/Walmart Unit Sales/calendar.csv')
sellprices = pd.read_csv('C:/Users/isach/Documents/Kaggle Competitions/Walmart Unit Sales/sell_prices.csv', sep=';')

#   3 states, 10 stores, 7 departamentos , 3 categorias

## CHANGING COLUMNS D_1 ... D_1919  TO ROWS   in sales_train

In [0]:
NO CORRER OTRA VEZ!!

# create an object reading your file 100 rows at a time
reader = pd.read_csv( 'C:/Users/isach/Documents/Kaggle Competitions/Walmart Unit Sales/sales_train_validation.csv',sep=';',  chunksize=1000)
#sep='\t', header=None,
# process each chunk at a time
for chunk in reader:
    result = chunk.melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'])
    # export the results into a new file
    result.to_csv( 'bigfile_melted.csv', mode='a' , sep=';') #header=None, ,)

## STARTING HERE

In [0]:
# Reading bifile_melted  of  sales_train
dtrain = pd.read_csv('C:/Users/isach/bigfile_melted.csv',sep=';')       #, chunksize=2000000 )

In [0]:
#dtrain = pd.DataFrame( df.get_chunk() )
#dtrain.drop(columns = ['id'], inplace=True)  #axis=1, 'Unnamed: 0',
dtrain.set_index(['id'], inplace=True)  #.stack(level=0)
dtrain.head()

,item_id,dept_id,cat_id,store_id,state_id,d,daily_sales
id,,,,,,,
HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [0]:
dtrain.rename(columns={'variable':'d', 'value':'daily_sales'}, inplace=True)
dtrain.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,daily_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


### creating dummy variables --- encoding

In [0]:
just_dummies = pd.get_dummies(dtrain[['dept_id','cat_id','store_id','state_id']],drop_first=True, prefix_sep='_')
just_dummies.head()

In [0]:
dtrain = pd.DataFrame(data=dtrain, columns=['item_id','variable','value'])
dtrain= pd.concat([dtrain, just_dummies], axis=1) 

#dtrain.drop(['dept_id','cat_id','store_id','state_id'], inplace=True, axis=1)

dtrain.head()

In [0]:
train.shape

In [0]:
30490*1913

58327370

In [0]:
dtrain.shape

In [0]:
#train.head(n=10)

In [0]:
calendar.head(n=10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [0]:
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [0]:
calendar.shape

(1969, 14)

In [0]:
sellprices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [0]:
sellprices.shape

(6841121, 4)

In [0]:
pd.crosstab(train['item_id'], columns='count')

col_0,count
item_id,
FOODS_1_001,10
FOODS_1_002,10
FOODS_1_003,10
FOODS_1_004,10
FOODS_1_005,10
...,...
HOUSEHOLD_2_512,10
HOUSEHOLD_2_513,10
HOUSEHOLD_2_514,10


In [0]:
pd.crosstab(train['state_id'], columns='count')

col_0,count
state_id,
CA,12196
TX,9147
WI,9147


In [0]:
pd.crosstab(train['dept_id'], columns='count')

col_0,count
dept_id,
FOODS_1,2160
FOODS_2,3980
FOODS_3,8230
HOBBIES_1,4160
HOBBIES_2,1490
HOUSEHOLD_1,5320
HOUSEHOLD_2,5150


In [0]:
pd.crosstab(train['store_id'], columns='count')

col_0,count
store_id,
CA_1,3049
CA_2,3049
CA_3,3049
CA_4,3049
TX_1,3049
TX_2,3049
TX_3,3049
WI_1,3049
WI_2,3049


In [0]:
pd.crosstab(train['cat_id'], columns='count')

col_0,count
cat_id,
FOODS,14370
HOBBIES,5650
HOUSEHOLD,10470


In [0]:
# train.shape  (30490, 1919)
train.isnull().sum()

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
           ..
d_1909      0
d_1910      0
d_1911      0
d_1912      0
d_1913      0
Length: 1919, dtype: int64

In [0]:
sellprices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [0]:
train.loc[:].mean(axis=1)

0        0.313643
1        0.257710
2        0.150549
3        1.718766
4        0.966545
           ...   
30485    0.534239
30486    0.376372
30487    0.894929
30488    0.366963
30489    0.694720
Length: 30490, dtype: float64

In [0]:
train.loc[:].sum(axis=1)

0         600
1         493
2         288
3        3288
4        1849
         ... 
30485    1022
30486     720
30487    1712
30488     702
30489    1329
Length: 30490, dtype: int64

# XGBOOST REGRESOR

In [0]:
from xgboost import XGBRegressor
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_cv, Y_cv)], 
    verbose=True, 
    early_stopping_rounds = 10)

In [0]:
feature_importances = pd.DataFrame({'col': columns,'imp':model.feature_importances_})
feature_importances = feature_importances.sort_values(by='imp',ascending=False)
px.bar(feature_importances,x='col',y='imp')

In [0]:
from itertools import product
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
grid.head()